In [16]:
import pandas as pd
import numpy as np

def merge_naics_data():
    """
    Merge California establishments data with AB1018 classification data.
    Matches 6-digit NAICS codes from AB1018 to 3-digit NAICS codes from California data.
    """
    
    # Read the California establishments data
    print("Reading California establishments data...")
    ca_data = pd.read_csv('extracted_california_data.txt')
    
    # Read the AB1018 classification data
    print("Reading AB1018 classification data...")
    ab1018_data = pd.read_csv('Claude_AB-1018_classification.csv')
    
    # Clean and prepare the data
    print("Preparing data for merge...")
    
    # Clean NAICS codes in California data - remove quotes and handle special cases
    ca_data['NAICS'] = ca_data['NAICS'].astype(str).str.replace('"', '').str.strip()
    
    # Create 3-digit NAICS prefix for California data (for matching)
    ca_data['NAICS_3digit'] = ca_data['NAICS'].apply(lambda x: x[:3] if len(x) >= 3 and x != '' else x)
    
    # Convert AB1018 NAICS to string and create 3-digit prefix
    ab1018_data['NAICS'] = ab1018_data['NAICS'].astype(str)
    ab1018_data['NAICS_3digit'] = ab1018_data['NAICS'].apply(lambda x: x[:3] if len(x) >= 3 else x)
    
    # Display some info about the datasets
    print(f"California data shape: {ca_data.shape}")
    print(f"AB1018 data shape: {ab1018_data.shape}")
    print(f"Unique 3-digit NAICS in CA data: {ca_data['NAICS_3digit'].nunique()}")
    print(f"Unique 3-digit NAICS in AB1018 data: {ab1018_data['NAICS_3digit'].nunique()}")
    
    # Perform the merge - keep all AB1018 rows, match with CA data where possible
    print("Performing merge...")
    merged_data = pd.merge(
        ab1018_data,
        ca_data,
        on='NAICS_3digit',
        how='left',
        suffixes=('_ab1018', '_ca')
    )
    
    # Rename columns for clarity
    merged_data = merged_data.rename(columns={
        'NAICS_ab1018': 'NAICS_6digit',
        'NAICS_ca': 'NAICS_ca_original'
    })
    
    # Reorder columns for better readability
    column_order = [
        'Business Type',
        'NAICS_6digit',
        'NAICS Description',
        'Regulation Type',
        'NAICS_3digit',
        'NAICS_ca_original',
        'Industry',
        'Third_Quarter_Payroll_Thousands',
        'Total_Businesses',
        'Size_0_4',
        'Size_5_9',
        'Size_10_19',
        'Size_20_49',
        'Size_50_99',
        'Size_100_249',
        'Size_250_499',
        'Size_500_999',
        'Size_1000_Plus'
    ]
    
    # Only include columns that exist in the merged data
    available_columns = [col for col in column_order if col in merged_data.columns]
    merged_data = merged_data[available_columns]
    
    print(f"Merged data shape: {merged_data.shape}")
    
    # Check for matches
    matched_rows = merged_data[merged_data['Industry'].notna()]
    unmatched_rows = merged_data[merged_data['Industry'].isna()]
    
    print(f"Successfully matched: {len(matched_rows)} rows")
    print(f"No CA data found for: {len(unmatched_rows)} rows")
    
    if len(unmatched_rows) > 0:
        print("\nUnmatched AB1018 entries (first 10):")
        print(unmatched_rows[['Business Type', 'NAICS_6digit', 'NAICS_3digit']].head(10))
    
    # Save the results
    output_filename = 'merged_naics_data.csv'
    merged_data.to_csv(output_filename, index=False)
    print(f"\nMerged data saved to: {output_filename}")
    
    # Display sample of merged data
    print("\nSample of merged data:")
    print(merged_data.head(10).to_string())
    
    return merged_data

# Alternative function if you want more control over the matching process
def merge_with_detailed_analysis():
    """
    Enhanced version with detailed analysis of the matching process.
    """
    
    # Read data
    ca_data = pd.read_csv('extracted_california_data.txt')
    ab1018_data = pd.read_csv('Claude_AB1018_classification.csv')
    
    # Clean NAICS codes
    ca_data['NAICS'] = ca_data['NAICS'].astype(str).str.replace('"', '').str.strip()
    ab1018_data['NAICS'] = ab1018_data['NAICS'].astype(str)
    
    # Create 3-digit prefixes
    ca_data['NAICS_3digit'] = ca_data['NAICS'].apply(lambda x: x[:3] if len(x) >= 3 and x != '' else x)
    ab1018_data['NAICS_3digit'] = ab1018_data['NAICS'].apply(lambda x: x[:3] if len(x) >= 3 else x)
    
    # Analysis of matching potential
    print("=== MATCHING ANALYSIS ===")
    ca_3digit_set = set(ca_data['NAICS_3digit'].unique())
    ab1018_3digit_set = set(ab1018_data['NAICS_3digit'].unique())
    
    print(f"CA data 3-digit NAICS codes: {sorted(ca_3digit_set)}")
    print(f"AB1018 data 3-digit NAICS codes: {sorted(ab1018_3digit_set)}")
    print(f"Intersection: {sorted(ca_3digit_set.intersection(ab1018_3digit_set))}")
    print(f"AB1018 codes not in CA data: {sorted(ab1018_3digit_set - ca_3digit_set)}")
    
    # Perform merge
    merged = pd.merge(ab1018_data, ca_data, on='NAICS_3digit', how='left', suffixes=('_ab1018', '_ca'))
    
    # Save detailed results
    merged.to_csv('detailed_merged_naics_data.csv', index=False)
    
    # Create summary report
    summary = ab1018_data.groupby('NAICS_3digit').agg({
        'Business Type': 'count',
        'Regulation Type': lambda x: ', '.join(x.unique())
    }).rename(columns={'Business Type': 'Count_AB1018_Entries'})
    
    # Add CA data info
    ca_summary = ca_data[ca_data['NAICS_3digit'].isin(ab1018_3digit_set)][['NAICS_3digit', 'Industry', 'Total_Businesses']].set_index('NAICS_3digit')
    summary = summary.join(ca_summary, how='left')
    
    summary.to_csv('naics_matching_summary.csv')
    print(f"\nDetailed analysis saved to: detailed_merged_naics_data.csv")
    print(f"Summary report saved to: naics_matching_summary.csv")
    
    return merged, summary

# Run the main merge function
if __name__ == "__main__":
    try:
        # Run basic merge
        merged_data = merge_naics_data()
        
        print("\n" + "="*50)
        print("MERGE COMPLETED SUCCESSFULLY!")
        print("="*50)
        
        # Uncomment the line below if you want the detailed analysis too
        # detailed_merged, summary = merge_with_detailed_analysis()
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        print("Please check that your input files exist and have the correct format.")

Reading California establishments data...
Reading AB1018 classification data...
Preparing data for merge...
California data shape: (107, 14)
AB1018 data shape: (92, 5)
Unique 3-digit NAICS in CA data: 106
Unique 3-digit NAICS in AB1018 data: 29
Performing merge...
Merged data shape: (92, 18)
Successfully matched: 78 rows
No CA data found for: 14 rows

Unmatched AB1018 entries (first 10):
                  Business Type NAICS_6digit NAICS_3digit
23   Public housing authorities       925110          925
25           Electric utilities       221122          221
26              Water utilities       221310          221
52              Courts using AI       922130          922
53        Probation departments       922140          922
54  District attorneys' offices       922120          922
66  Social services departments       923130          923
67                 Tax agencies       921130          921
68         Immigration services       928120          928
69         Unemployment offic

In [17]:
import pandas as pd
import numpy as np

def consolidate_naics_data(input_file='merged_naics_data.csv', output_file='consolidated_naics_data.csv'):
    """
    Consolidate merged NAICS data by grouping rows with the same NAICS_ca_original.
    Combines Business Type information and removes specified columns.
    """
    
    print("Reading merged NAICS data...")
    df = pd.read_csv(input_file)
    
    print(f"Input data shape: {df.shape}")
    print(f"Unique NAICS_ca_original codes: {df['NAICS_ca_original'].nunique()}")
    
    # Check for duplicates
    duplicate_counts = df['NAICS_ca_original'].value_counts()
    duplicates = duplicate_counts[duplicate_counts > 1]
    
    print(f"NAICS codes with multiple entries: {len(duplicates)}")
    if len(duplicates) > 0:
        print("Top duplicate NAICS codes:")
        print(duplicates.head(10))
    
    # Define aggregation functions for each column
    agg_functions = {
        'Business Type': lambda x: ' | '.join(sorted(set(str(val) for val in x if pd.notna(val)))),
        'Regulation Type': lambda x: ' | '.join(sorted(set(str(val) for val in x if pd.notna(val)))),
        'Industry': 'first',  # Should be the same for all rows with same NAICS_ca_original
        'Third_Quarter_Payroll_Thousands': 'first',
        'Total_Businesses': 'first',
        'Size_0_4': 'first',
        'Size_5_9': 'first',
        'Size_10_19': 'first',
        'Size_20_49': 'first',
        'Size_50_99': 'first',
        'Size_100_249': 'first',
        'Size_250_499': 'first',
        'Size_500_999': 'first',
        'Size_1000_Plus': 'first'
    }
    
    print("\nConsolidating data by NAICS_ca_original...")
    
    # Group by NAICS_ca_original and aggregate
    consolidated = df.groupby('NAICS_ca_original').agg(agg_functions).reset_index()
    
    # Rename the Business Type column to indicate it contains combined information
    consolidated = consolidated.rename(columns={'Business Type': 'Combined_Business_Types'})
    
    # Remove the specified columns (they don't exist in grouped data, but let's be explicit)
    columns_to_remove = ['NAICS_6digit', 'NAICS Description', 'NAICS_3digit']
    # These columns won't exist after groupby, but if we had kept them, we'd remove them here
    
    # Reorder columns for better readability
    column_order = [
        'NAICS_ca_original',
        'Industry',
        'Combined_Business_Types',
        'Regulation Type',
        'Third_Quarter_Payroll_Thousands',
        'Total_Businesses',
        'Size_0_4',
        'Size_5_9',
        'Size_10_19',
        'Size_20_49',
        'Size_50_99',
        'Size_100_249',
        'Size_250_499',
        'Size_500_999',
        'Size_1000_Plus'
    ]
    
    # Only include columns that exist
    available_columns = [col for col in column_order if col in consolidated.columns]
    consolidated = consolidated[available_columns]
    
    print(f"Consolidated data shape: {consolidated.shape}")
    
    # Show some statistics about the consolidation
    print(f"\nConsolidation Results:")
    print(f"Original rows: {len(df)}")
    print(f"Consolidated rows: {len(consolidated)}")
    print(f"Rows reduced by: {len(df) - len(consolidated)}")
    
    # Show examples of combined business types
    print("\nExamples of combined business types:")
    for idx, row in consolidated.head(10).iterrows():
        if ' | ' in str(row['Combined_Business_Types']):
            print(f"NAICS {row['NAICS_ca_original']}: {row['Combined_Business_Types']}")
    
    # Save the consolidated data
    consolidated.to_csv(output_file, index=False)
    print(f"\nConsolidated data saved to: {output_file}")
    
    # Display sample of consolidated data
    print("\nSample of consolidated data:")
    print(consolidated.head().to_string())
    
    return consolidated

def create_detailed_analysis(consolidated_df):
    """
    Create additional analysis of the consolidated data.
    """
    
    print("\n" + "="*60)
    print("DETAILED ANALYSIS")
    print("="*60)
    
    # Analyze regulation types
    reg_type_counts = consolidated_df['Regulation Type'].value_counts()
    print("\nRegulation Type Distribution:")
    print(reg_type_counts)
    
    # Analyze industries with most business types
    consolidated_df['Num_Business_Types'] = consolidated_df['Combined_Business_Types'].apply(
        lambda x: len(str(x).split(' | ')) if pd.notna(x) else 0
    )
    
    top_diverse = consolidated_df.nlargest(10, 'Num_Business_Types')[
        ['NAICS_ca_original', 'Industry', 'Num_Business_Types', 'Combined_Business_Types']
    ]
    
    print("\nTop 10 NAICS codes with most business types:")
    print(top_diverse.to_string())
    
    # Analyze business size distribution
    size_columns = ['Size_0_4', 'Size_5_9', 'Size_10_19', 'Size_20_49', 
                   'Size_50_99', 'Size_100_249', 'Size_250_499', 'Size_500_999', 'Size_1000_Plus']
    
    # Calculate total businesses for AI-related industries only
    ai_related = consolidated_df[consolidated_df['Combined_Business_Types'].notna()]
    
    if len(ai_related) > 0:
        print(f"\nAI-related industries: {len(ai_related)} NAICS codes")
        print(f"Total businesses in AI-related codes: {ai_related['Total_Businesses'].sum():,}")
        print(f"Total payroll in AI-related codes: ${ai_related['Third_Quarter_Payroll_Thousands'].sum() * 1000:,}")
    
    # Save detailed analysis
    analysis_df = consolidated_df[['NAICS_ca_original', 'Industry', 'Num_Business_Types', 
                                  'Combined_Business_Types', 'Regulation Type', 'Total_Businesses']]
    analysis_df.to_csv('detailed_analysis.csv', index=False)
    print(f"\nDetailed analysis saved to: detailed_analysis.csv")

# Alternative function for custom grouping logic
def advanced_consolidation(input_file='merged_naics_data.csv'):
    """
    Advanced consolidation with more sophisticated business type combination.
    """
    
    df = pd.read_csv(input_file)
    
    def smart_combine_business_types(group):
        """Custom function to intelligently combine business types."""
        business_types = group['Business Type'].dropna().unique()
        
        # Sort and combine, removing duplicates
        unique_types = sorted(set(str(bt).strip() for bt in business_types if str(bt).strip()))
        
        # Group similar types (you can customize this logic)
        combined = ' | '.join(unique_types)
        
        return combined
    
    def combine_regulation_types(group):
        """Combine regulation types, prioritizing 'Both' if present."""
        reg_types = group['Regulation Type'].dropna().unique()
        
        if 'Both' in reg_types:
            return 'Both'
        elif len(reg_types) > 1:
            return ' | '.join(sorted(reg_types))
        else:
            return reg_types[0] if len(reg_types) > 0 else 'Unknown'
    
    # Custom groupby with advanced logic
    result = df.groupby('NAICS_ca_original').apply(
        lambda group: pd.Series({
            'Industry': group['Industry'].iloc[0],
            'Combined_Business_Types': smart_combine_business_types(group),
            'Regulation_Type_Combined': combine_regulation_types(group),
            'Business_Type_Count': group['Business Type'].nunique(),
            'Third_Quarter_Payroll_Thousands': group['Third_Quarter_Payroll_Thousands'].iloc[0],
            'Total_Businesses': group['Total_Businesses'].iloc[0],
            'Size_0_4': group['Size_0_4'].iloc[0],
            'Size_5_9': group['Size_5_9'].iloc[0],
            'Size_10_19': group['Size_10_19'].iloc[0],
            'Size_20_49': group['Size_20_49'].iloc[0],
            'Size_50_99': group['Size_50_99'].iloc[0],
            'Size_100_249': group['Size_100_249'].iloc[0],
            'Size_250_499': group['Size_250_499'].iloc[0],
            'Size_500_999': group['Size_500_999'].iloc[0],
            'Size_1000_Plus': group['Size_1000_Plus'].iloc[0]
        })
    ).reset_index()
    
    result.to_csv('advanced_consolidated_naics_data.csv', index=False)
    print("Advanced consolidation saved to: advanced_consolidated_naics_data.csv")
    
    return result

# Main execution
if __name__ == "__main__":
    try:
        # Run basic consolidation
        consolidated_data = consolidate_naics_data()
        
        # Run detailed analysis
        create_detailed_analysis(consolidated_data)
        
        # Uncomment for advanced consolidation
        # advanced_data = advanced_consolidation()
        
        print("\n" + "="*50)
        print("CONSOLIDATION COMPLETED SUCCESSFULLY!")
        print("="*50)
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

Reading merged NAICS data...
Input data shape: (92, 18)
Unique NAICS_ca_original codes: 25
NAICS codes with multiple entries: 12
Top duplicate NAICS codes:
NAICS_ca_original
541    34
92     11
524     6
522     6
561     5
611     3
485     3
621     3
813     2
531     2
Name: count, dtype: int64

Consolidating data by NAICS_ca_original...
Consolidated data shape: (25, 15)

Consolidation Results:
Original rows: 92
Consolidated rows: 25
Rows reduced by: 67

Examples of combined business types:
NAICS 22: Electric utilities | Water utilities
NAICS 485: Gig economy platforms (Uber, DoorDash) | Public transit agencies | Ride-sharing companies
NAICS 517: Internet service providers | Telecommunications companies
NAICS 522: Banks using AI | Credit unions using AI | Lending platforms | Mortgage companies | Mortgage lenders | Payment processors with fraud detection

Consolidated data saved to: consolidated_naics_data.csv

Sample of consolidated data:
  NAICS_ca_original                        